## Objective
In this notebook, I...

1. Get the GPS points for every ATM address uing the Google Maps and OpenStreetMap geocode services
2. Calculate the distance between the two GPS points and flag instances where the distance is more than X
3. Create a map of all the points using Folium

## Python packages used and references

* I use the google API for which I need an API key. I used code from the "create basic google map" notebook for this.
* I use the [geopy package](https://geopy.readthedocs.io/en/stable/). The geopy package provides a consistent, simple python interface for several different geocoding packages including Google Maps and OSM (Nominatum).
* I used this [blog post](https://thejoeblankenship.com/blogs/geopy_folium/geopy_folium.html)
* I used the blog posts [here](https://towardsdatascience.com/geocode-with-python-161ec1e62b89) 
* I also use folium to map the output.

In [38]:
import os
import geopy
import pandas as pd
import folium
%matplotlib inline

In [39]:
# get Google Maps API key
os.chdir("/mnt/c/Users/dougj/Documents/GIS")
with open('google_api_key2.txt') as f:
    api_key = f.readline()
    f.close

In [57]:
# import ATM address data from Robert
os.chdir("/mnt/c/Users/dougj/Documents/Philippines cash transfer/Docs from Robert")
atm_locs = pd.read_excel("LBP ATM locations clean.xlsx")

In [88]:
# Get first address from dataframe
first_address = atm_locs["ADDRESS"][7]
first_address = first_address + ", Philippines"

# Use google maps API to get location
google_locator = geopy.GoogleV3(api_key=api_key)
google_first_location = google_locator.geocode(first_address)

# Use OSM API to get location
osm_locator = geopy.Nominatim(user_agent="test_app")
osm_first_location = osm_locator.geocode(first_address)

# Seems like OSM doesn't return anything.  Just using Google for now, but worth trying ESRI's paid geocode service later

In [87]:
google_first_location

Location(Constitution Hills, Batasan Pambansa Complex Diliman, Quezon City, Metro Manila, Philippines, (14.6921848, 121.098077, 0.0))

In [62]:
# Store the first 10 rows in a new dataframe
df = atm_locs.head(10)
df.loc[:, 'ADDRESS']=  df["ADDRESS"]+ ", Philippines"

In [89]:
# Get GPS points for the first few rows the Google Maps API
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(google_locator.geocode, min_delay_seconds=1)
df['location'] = df['ADDRESS'].apply(google_locator.geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

/home/doug/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/doug/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [104]:
# Plot the map with all of the points
phil_map = folium.Map(location = [df.point[0][0], df.point[0][1]], tiles='OpenStreetMap', zoom_start=12)

feature_group = folium.FeatureGroup("Locations")
for index, row in df.iterrows():
    feature_group.add_child(folium.Marker(location=[row.point[0],row.point[1]],popup=row.GROUP))
phil_map.add_child(feature_group)